Tokenization can have a significant impact on the quality of models. In this task, you will need to reduce the tokenizer to reduce the size of the model while maintaining quality. You will solve the problem of sentiment analysis. In the first part of the notebook, we implement a baseline, which measures the quality of the original model. In the second part, you need to reduce the tokenizer by removing at least 50% of tokens from it, reinitialize the embedding layer of BERT model and retrain the model. The classification quality should decrease by no more than 2% in terms of the F-measure, compared to the original model.

### Baseline

In [1]:
!pip install transformers datasets evaluate accelerate tokenizers -q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    BertTokenizerFast
)
from datasets import load_dataset
import evaluate

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 1. Загрузка данных
dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(seed=42).select(range(5000))
test_data = dataset["test"].shuffle(seed=42).select(range(1000))

In [5]:
# 2. Исходный токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

In [6]:
# 3. Обучение исходной модели с выводом Accuracy и F1

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
).cuda()

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to=['tensorboard'],
    bf16=True,
)

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    return {"eval_accuracy": acc["accuracy"], "eval_f1": f1["f1"]}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.377587,0.845000,0.858447
2,0.304400,0.365902,0.882000,0.887619
3,0.304400,0.418461,0.895000,0.895314


TrainOutput(global_step=939, training_loss=0.21991622486053564, metrics={'train_runtime': 100.8061, 'train_samples_per_second': 148.801, 'train_steps_per_second': 9.315, 'total_flos': 1973332915200000.0, 'train_loss': 0.21991622486053564, 'epoch': 3.0})

In [7]:
# 4. Оценка исходной модели
original_metrics = trainer.evaluate()
original_accuracy = original_metrics["eval_accuracy"]
original_f1 = original_metrics["eval_f1"]

print(f"Исходная точность: {original_accuracy:.4f}")
print(f"Исходный F1: {original_f1:.4f}")

Исходная точность: 0.8950
Исходный F1: 0.8953


### Task 1

Remove at least 50% of tokens from original tokenizer. Reinitialize new tokenizer. Give to new tokenizer 'new_tokenizer' name.

In [8]:
freq_dict = {}
for txt_row in train_data["text"]:
    split_result = tokenizer.tokenize(txt_row)
    for elem in split_result:
        freq_dict[elem] = freq_dict.get(elem, 0) + 1

sorted_elems = sorted(freq_dict.items(), key=lambda x: -x[1])
base_vocab_len = len(tokenizer.vocab)
shrinked_vocab_len = base_vocab_len // 2

reserved = list(tokenizer.all_special_tokens)
filtered_symbols = reserved.copy()

for item in sorted_elems:
    if item[0] not in filtered_symbols:
        filtered_symbols.append(item[0])
    if len(filtered_symbols) >= shrinked_vocab_len:
        break

vocab = "shrunk_vocabulary.txt"
with open(vocab, "w", encoding="utf-8") as output_file:
    output_file.write("\n".join(filtered_symbols))

new_tokenizer = BertTokenizerFast(vocab_file=vocab, do_lower_case=True)

print(f"\nРазмеры словарей:")
print(f"Исходный: {len(tokenizer.vocab)}")
print(f"Новый: {len(new_tokenizer.vocab)}")
print(f"Удалено токенов: {len(tokenizer.vocab) - len(new_tokenizer.vocab)}")
print(f"Удалено токенов в %: {(len(tokenizer.vocab) - len(new_tokenizer.vocab)) * 100 / len(tokenizer.vocab)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (934 > 512). Running this sequence through the model will result in indexing errors



Размеры словарей:
Исходный: 30522
Новый: 15261
Удалено токенов: 15261
Удалено токенов в %: 50.0


### Task 2:

Initialize new Embedding layer of BERT model according to new tokenizer.
Retrain model on classification task. The classification quality should decrease by no more than 2% in terms of the F-measure, compared to the original model.

In [9]:
# 7. Переинициализация модели с новым словарем

new_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    ignore_mismatched_sizes=True
).cuda()

old_embeddings = model.get_input_embeddings().weight.data.clone()
new_embeddings = new_model.get_input_embeddings().weight.data

for token, new_idx in new_tokenizer.vocab.items():
    if token in tokenizer.vocab:
        old_idx = tokenizer.vocab[token]
        new_embeddings[new_idx] = old_embeddings[old_idx]

def tokenize_function(examples):
    return new_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset_new = train_data.map(tokenize_function, batched=True)
test_dataset_new = test_data.map(tokenize_function, batched=True)

training_args_new = TrainingArguments(
    output_dir="./results_new",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.001,
    logging_dir="./logs_new",
    report_to=['tensorboard'],
    bf16=True,
    lr_scheduler_type="cosine",
    warmup_steps=450,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
# 8. Обучение с новым токенизатором

trainer_new = Trainer(
    model=new_model,
    args=training_args_new,
    train_dataset=train_dataset_new,
    eval_dataset=test_dataset_new,
    compute_metrics=compute_metrics,
)

trainer_new.train()

new_metrics = trainer_new.evaluate()
new_f1 = new_metrics.get("eval_f1")
new_accuracy = new_metrics.get("eval_accuracy", None)

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.508600,0.310814,0.882000,0.881764
2,0.303400,0.404601,0.885000,0.886026
3,0.196000,0.442567,0.891000,0.890672


In [11]:
# 9. Результаты
print("\nРезультаты сравнения:")
print(f"Исходный F1: {original_f1:.4f}")
print(f"F1 с новым словарем: {new_f1:.4f}")
print(f"Разница F1: {original_f1 - new_f1:.4f}")
print(f"Экономия памяти: {(len(tokenizer.vocab) - len(new_tokenizer.vocab))/len(tokenizer.vocab):.2%}")


Результаты сравнения:
Исходный F1: 0.8953
F1 с новым словарем: 0.8907
Разница F1: 0.0046
Экономия памяти: 50.00%
